In [2]:
import pandas as pd
import re

# 主資料合併

In [342]:
# 合併所有zipcode的房屋資料
c = 1
zipcode = []
df = pd.read_csv('zipcode_data/98001.csv')
for i in range(98002, 98289):
    try:
        add = pd.read_csv('zipcode_data/%s.csv' % str(i))
        add.columns = ['mls', 'housebedroom', 'housebathroom', 'sqft', 'address', 'soldout',
                       'walkscore', 'transitscore', 'school1', 'school2', 'school3',
                       'houseyear', 'lot', 'pt', 'ta', 'houseprice']
        df = df.append(add,ignore_index=True)
        c += 1
        zipcode.append(i)
    except:
        None
        
for i in range(len(df)):
    if type(df.iloc[i,12]) == str:
        df.iloc[i,12] = df.iloc[i,12].replace(',','')
df['lot'] = df['lot'].astype(float)

In [343]:
df.isnull().sum()

mls              2
housebedroom     0
housebathroom    0
sqft             0
address          0
soldout          0
walkscore        0
transitscore     0
school1          0
school2          0
school3          0
houseyear        0
lot              0
pt               0
ta               0
houseprice       0
dtype: int64

In [344]:
df = df.reindex(columns = ['mls', 'address', 'soldout','housebedroom', 'housebathroom', 'sqft',
       'walkscore', 'transitscore', 'school1', 'school2', 'school3',
       'houseyear', 'lot', 'pt', 'ta', 'houseprice'])

In [345]:
# 找出不是 WA 的地址並刪除 
count = 0
drop = []
for i in df['address']:
    if 'WA' not in i:
        drop.append(count)
    count += 1 
print(df['address'][drop])
df.drop(drop, axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

26434              12326 94th, Avenue, NE 98034
57062    106 Princess Ave, Greenville, SC 29611
75551         14523 37th Ave, Seattle, NE 98155
Name: address, dtype: object


In [347]:
# 新增zipcode欄位
c=0
zipcodeList = []
for i in df['address']:
    zipcode = re.search('WA 98[0-9]+', i).group().replace('WA ', '')
    zipcodeList.append(zipcode)
df['zipcode'] = zipcodeList

In [352]:
# 屋齡計算
c = 0
l = []
for i in df['soldout']:
    if '$' in  i:
        l.append(c)       
    c += 1 


In [350]:
df.drop(l, axis=0, inplace=True)

In [353]:
for i in df['soldout']:
    if '$' in  i:
        print(i)
        #l.append(c)       
    c += 1 

In [354]:
df['houseyear'] = df['soldout'].str.replace('[0-9]{1,2}/[0-9]{1,2}/',  '').astype(int) - df['houseyear']

<ipython-input-354-3f445bdfd77a>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['houseyear'] = df['soldout'].str.replace('[0-9]{1,2}/[0-9]{1,2}/',  '').astype(int) - df['houseyear']


In [355]:
# 刪除 mls
df.drop(['mls'], axis=1, inplace=True)

In [356]:
df = df.reindex(columns = ['address', 'soldout', 'zipcode', 'housebedroom', 'housebathroom', 'sqft',
       'walkscore', 'transitscore', 'school1', 'school2', 'school3',
       'houseyear', 'lot', 'pt', 'ta', 'houseprice'])
df['zipcode'] = df['zipcode'].astype(int)
df.reset_index(drop=True, inplace=True)

In [358]:
df.to_csv('zipcode_data/zillowData.csv', index=0)

# 外部資料與主資料結合

In [29]:
df = pd.read_csv('zipcode_data/zillowData.csv')
outsideCh = pd.read_excel('外部資料.xlsx', sheet_name='中文')
outsideEn = pd.read_excel('外部資料.xlsx', sheet_name='英文')
df3 = pd.read_csv('taxrate.csv')
exchangeRate = pd.read_csv('exchangeRate.csv')

In [30]:
df3.columns = ['zipcode', 'salesTaxRate']

In [31]:
outsideCh.drop(['Unnamed: 23', 'Unnamed: 24'],axis=1, inplace=True)

In [32]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [33]:
outsideEn.isnull().sum()

zipcode                                  0
population density                       0
housing units                            0
land area                                0
median household income                  0
households with kids                     0
year housing was built before 1970       0
year housing was built after 1970        0
household income < 25k                   0
household income 25k~60K                 0
hosuehold income 60k~150k                0
household income > 150k                  0
~25k                                     0
25k~60k                                  0
60k~150k                                 0
150k~                                    0
means of transportation to work (Car)    0
educational attainment (~high)           0
associate                                0
bachelor                                 0
master                                   0
dtype: int64

In [34]:
# 主資料與外部資料合併
allData = pd.merge(left=df,
                   right=outsideEn,
                   how='inner',
                   on='zipcode')
allData = pd.merge(left=allData,
                   right=outsideCh,
                   how='inner',
                   on='zipcode')
allData = pd.merge(left=allData,
                   right=df3,
                   how='left',
                   on='zipcode')
allData = pd.merge(left=allData,
                   right=exchangeRate,
                   how='left',
                   on='soldout')

In [40]:
# 2013年以前的匯率沒資料 用平均值補值
allData['ExchangeRate'].fillna(allData['ExchangeRate'].mean(), inplace=True)

In [41]:
allData = allData.reindex(columns = ['address', 'soldout', 'zipcode', 'housebedroom', 'housebathroom',
       'sqft', 'walkscore', 'transitscore', 'school1', 'school2', 'school3',
       'houseyear', 'lot', 'pt', 'ta','population density',
       'housing units ', 'land area', 'median household income ',
       'households with kids ', 'year housing was built before 1970',
       'year housing was built after 1970', 'household income < 25k',
       'household income 25k~60K', 'hosuehold income 60k~150k',
       'household income > 150k', '~25k', '25k~60k', '60k~150k', '150k~',
       'means of transportation to work (Car)',
       'educational attainment (~high)', 'associate', 'bachelor', 'master',
       '人口數', '性別男', '性別女', '性別比', '白人', '黑人', '西班牙', '亞洲', '夏威夷', '印度裔', '其他',
       '白人比率', '黑人.1', '西班牙裔', '亞洲.1', '印度裔.1', '其他.1', '年齡中位數', '男性年齡中位數',
       '女性年齡中位數', '每戶人數', '每戶平均收入', 'salesTaxRate', 'ExchangeRate', 'houseprice'])


In [42]:
# 欄位名稱更改
allData.columns = [ 'address', 'soldout', 'zipcode', 'bedroom', 'bathroom', 'sqft',
       'walkscore', 'transitscore', 'grade1_5', 'grade6_8', 'grade9_12',
       'houseyear', 'lot', 'propertyTax', 'totalAssessment',
       'populationDensity', 'housingUnits ', 'landArea', 'medianIncome ',
       'withKids ', 'before_1970', 'after_1970', 'income_25k', 'income25k_60K',
       'income60k_150k', 'income150k_', '_25k', '25k_60k', '60k_150k', '150k_',
       'CommuteByCar_mean', 'highschool', 'associate', 'bachelor', 'master',
       'population', 'male', 'female', 'sexRatio', 'White', 'African',
       'Latino', 'Asian', 'Hawaiian', 'Indian', 'Others', 'WhiteRatio',
       'AfricanRatio', 'LatinoRatio', 'AsianRatio', 'IndianRatio',
       'OthersRatio', 'MedianAge', 'MaleMedianAge', 'FemaleMedianAge',
       'People_house', 'AverageIncome_house',  'salesTaxRate', 'ExchangeRate', 'houseprice']

In [21]:
df.to_csv('allData.csv', index=0)